In [24]:
# Initial modules
import numpy as np
import pandas as pd
import sklearn

In [25]:
#load data
traindata= pd.read_csv('C:/Users/amatu/Documents/train_imperson_without4n7_balanced_data.csv')
testdata= pd.read_csv('C:/Users/amatu/Documents/test_imperson_without4n7_balanced_data.csv')

In [26]:
# Separate X and Y

X_train, Y_train = traindata.loc[:, traindata.columns != '155'], traindata['155']
X_test, Y_test = testdata.loc[:, testdata.columns != '155'], testdata['155']


In [27]:
#Preprocessing pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline


preprocessing_pipeline = Pipeline([
    ('zero variance', VarianceThreshold()),
    ('scale 0_1', MinMaxScaler()),
    ('top20 features', SelectKBest(chi2, k=20))])



In [28]:
X_train_ready = preprocessing_pipeline.fit_transform(X_train, Y_train)

In [29]:
#using Adaboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

num_folds = 10
seed = 7
kfolds = KFold(n_splits=num_folds, random_state=seed)
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X_train_ready, Y_train, cv=kfolds)
scores.mean(), scores.var()


(0.9870775203626522, 0.0012421924117970102)

In [30]:
#Adaboost trained on training set and fit to test set
clf1 = AdaBoostClassifier(n_estimators=100)
clf1.fit(X_train_ready, Y_train)
X_test_ready =  preprocessing_pipeline.transform(X_test)
scores = cross_val_score(clf1, X_test_ready,Y_test, cv=10)
scores.mean()

0.9953685258964142

In [31]:
from sklearn.metrics import classification_report

prediction = clf1.predict(X_test_ready)
report = classification_report(Y_test, prediction)
print(report)

              precision    recall  f1-score   support

           0       0.52      1.00      0.68     20079
           1       0.99      0.07      0.14     20079

    accuracy                           0.54     40158
   macro avg       0.75      0.54      0.41     40158
weighted avg       0.75      0.54      0.41     40158

